In [22]:
from datetime import datetime, timezone
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from typing import Tuple



In [3]:
ticker = yf.Ticker("NVDA")
expiry = ticker.options[1]             # one expiry for now
chain = ticker.option_chain(expiry)
spot = ticker.info['currentPrice'] 

calls = chain.calls

## B. Heston model

&rarr; make variance stochastic and correlated with the asset:


### Risk-neutral dynamics (the version you price with)

Under the risk-neutral measure $\mathbb{Q}$ (so drift is $r - q$):

$$
\frac{dS_t}{S_t} = (r - q)\,dt + \sqrt{v_t}\,dW_t^{(1)}
$$

$$
dv_t = \kappa(\theta - v_t)\,dt + \xi\sqrt{v_t}\,dW_t^{(2)}
$$

with correlation

$$
dW_t^{(1)}\, dW_t^{(2)} = \rho\, dt.
$$

### Parameters (to calibrate..)

- $v_0$: initial variance at $t = 0$ (so initial vol $\approx \sqrt{v_0}$)
- $\theta$: long-run variance level
- $\kappa$: mean reversion speed (how fast $v_t$ returns to $\theta$)
- $\xi$: "vol of vol" (variance volatility)
- $\rho$: correlation between spot and variance shocks (typically negative in equities $\rightarrow$ skew)

Plus market inputs: $S_0$, $r$, $q$.



In [4]:

T = 1.0  #years       
N = 2000         
rho = 0.7        # correlation in [-1, 1]
seed = 42

rng = np.random.default_rng(seed)
dt = T / N
t = np.linspace(0, T, N + 1)

# independent Brownian increments
dB1 = np.sqrt(dt) * rng.standard_normal(N)
dB2 = np.sqrt(dt) * rng.standard_normal(N)


# correlated increments
dW1 = dB1
dW2 = rho * dB1 + np.sqrt(1 - rho**2) * dB2

W1 = np.concatenate([[0.0], np.cumsum(dW1)])
W2 = np.concatenate([[0.0], np.cumsum(dW2)])


fig = go.Figure()

# paths
fig.add_trace(go.Scatter(x=t, y=W1, mode="lines", name="W1"))
fig.add_trace(go.Scatter(x=t, y=W2, mode="lines", name="W2"))

fig.update_layout(
    title=f"Correlated Wiener Processes (rho = {rho})",
    xaxis_title="t",
    yaxis_title="W(t)",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=dW1, y=dW2, mode="markers", name="(dW1, dW2)",
    marker=dict(size=4, opacity=0.5)
))
fig2.update_layout(
    title="Increment Scatter (tilt shows correlation)",
    xaxis_title="dW1",
    yaxis_title="dW2",
    template="plotly_white"
)
fig2.show()


In [5]:
# def heston_cf(u, S0, v0, r, q, T, kappa, theta, sigma, rho):   
#     """
#     Returns Characteristic function of X_T = ln(S_T):
#     phi(u) = E[ exp(i u ln S_T) ] under risk-neutral measure ( using affine/Riccati ODEs , 
#     Feynman–Kac theorem to link PDE to an expectation of a stochastic process)
#     """
#     if np.any(np.isclose(sigma, 0.0)):
#         raise ValueError(f"sigma too small / zero: {sigma}")

#     u = np.asarray(u, dtype=np.complex128)
#     x0 = np.log(S0)
#     a = kappa* theta
#     b = kappa

#     iu = 1j*u
#     d = np.sqrt((rho * sigma * iu - b) ** 2 + sigma**2 * (iu + u**2))
#     # g = (b - rho * sigma * iu - d) / (b - rho * sigma * iu + d)
#     g = (b - rho * sigma * iu + d) / (b - rho * sigma * iu - d)


#     exp_neg_dT = np.exp(-d * T)

#     # Avoid problem if (1 - g*exp(-dT)) is near zero
#     one_minus_g_exp = 1.0 - g * exp_neg_dT
#     one_minus_g = 1.0 - g

#     D = ((b - rho * sigma * iu - d) / sigma**2) * ((1.0 - exp_neg_dT) / one_minus_g_exp)

#     C = (r - q) * iu * T + (a / sigma**2) * (
#         (b - rho * sigma * iu - d) * T - 2.0 * np.log(one_minus_g_exp / one_minus_g)
#     )

#     return np.exp(C + D * v0 + iu * x0)

#a stable version 
def heston_cf(u, S0, v0, r, q, T, kappa, theta, sigma, rho):
    u = np.asarray(u, dtype=np.complex128)

    x0 = np.log(S0)
    a = kappa * theta
    b = kappa

    iu = 1j * u

    d = np.sqrt(
        (rho * sigma * iu - b)**2
        + sigma**2 * (iu + u**2)
    )

    # Little Heston Trap (THIS is the key)
    g = (b - rho * sigma * iu + d) / (b - rho * sigma * iu - d)

    exp_dT = np.exp(d * T)

    C = (r - q) * iu * T + (a / sigma**2) * (
        (b - rho * sigma * iu + d) * T
        - 2.0 * np.log((1 - g * exp_dT) / (1 - g))
    )

    D = ((b - rho * sigma * iu + d) / sigma**2) * (
        (1 - exp_dT) / (1 - g * exp_dT)
    )

    return np.exp(C + D * v0 + iu * x0)









      

In [6]:
v0    = 0.04
kappa = 2.0
theta = 0.04
sigma = 0.6
rho   = -0.6
r = 0.05
q = 0.0
S0 = 100


In [7]:
u = np.array([0.1, 1.0, 5.0])

phi_u = heston_cf(
    u=u,
    S0=S0,
    r=r,
    q=q,
    sigma=sigma,
    T=T,
    v0=v0,
    kappa=kappa,
    theta=theta,
    rho=rho
)

print(phi_u)


[ 0.89428577+0.44699983j -0.07301462-0.97574165j -0.13345238-0.66053921j]


In [8]:


def _simpson(y: np.ndarray, x: np.ndarray) -> float:
    """
    Simpson's rule for evenly spaced grid.
    Requires odd number of points.
    """
    n = len(x)
    if n < 3 or (n % 2 == 0):
        raise ValueError("Simpson requires an odd number of points >= 3.")
    h = x[1] - x[0]
    return (h / 3.0) * (y[0] + y[-1] + 4.0 * y[1:-1:2].sum() + 2.0 * y[2:-2:2].sum())

$$
C = S_0 e^{-qT} P_1 - K e^{-rT} P_2.
$$

- $P_2$ is basically the risk-neutral probability that the option finishes in the money:

  $$
  P_2 = \mathbb{Q}(S_T > K).
  $$

- $P_1$ is like the same event but under a stock-numeraire (share) measure (sometimes called $\mathbb{Q}^S$):

  $$
  P_1 = \mathbb{Q}^{S}(S_T > K).
  $$

$$
P_2=\frac{1}{2}+\frac{1}{\pi}\int_{0}^{\infty}\Re\!\left(\frac{e^{-iu\ln K}\,\phi(u)}{iu}\right)\,du
$$

$$
P_1=\frac{1}{2}+\frac{1}{\pi}\int_{0}^{\infty}\Re\!\left(\frac{e^{-iu\ln K}\,\phi(u-i)}{iu\,\phi(-i)}\right)\,du
$$


In [9]:



def heston_call_price_cf(
    S0: float,
    K: float,
    v0: float,
    r: float,
    q: float,
    T: float,
    kappa: float,
    theta: float,
    sigma: float,
    rho: float,
    u_max: float = 200.0,
    n_u: int = 4001,
) -> float:
    """
    European call via Heston semi-closed form:
        C = S0*e^{-qT}*P1 - K*e^{-rT}*P2
    with P1, P2 computed by Fourier integrals.

    Integration is truncated at u_max and evaluated with Simpson.
    """
    if n_u % 2 == 0:
        n_u += 1  # make it odd for Simpson

    u = np.linspace(1e-10, u_max, n_u) # avoid division by zero
    lnK = np.log(K)

    # P2 integrand uses phi(u)
    phi_u = heston_cf(u, S0, v0, r, q, T, kappa, theta, sigma, rho)
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u) )
    integrand_P2 = np.nan_to_num(integrand_P2)
    
    # P1 uses phi(u - i) / phi(-i)
    phi_um_i = heston_cf(
        u=u - 1j,
        S0=S0,
        r=r,
        q=q,
        sigma=sigma,
        T=T,
        v0=v0,
        kappa=kappa,
        theta=theta,
        rho=rho
    )

    phi_minus_i = heston_cf(
        u=np.array([-1j]),
        S0=S0,
        r=r,
        q=q,
        sigma=sigma,
        T=T,
        v0=v0,
        kappa=kappa,
        theta=theta,
        rho=rho
    )[0]

    integrand_P1 = np.real(np.exp(-1j * u * lnK) * (phi_um_i / phi_minus_i) / (1j * u))

    P2 = 0.5 + (1.0 / np.pi) * _simpson(integrand_P2, u)
    P1 = 0.5 + (1.0 / np.pi) * _simpson(integrand_P1, u)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    return float(np.real(call))




In [10]:
S0 = 100.0
K  = 100.0
r  = 0.05
q  = 0.0
T  = 1.0

v0    = 0.04
kappa = 2.0
theta = 0.04
sigma = 0.5
rho   = -0.7

price = heston_call_price_cf(S0, K, v0, r, q, T, kappa, theta, sigma, rho)
print("Heston price:", price)


Heston price: nan


/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:51: RuntimeWarning:

divide by zero encountered in divide

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:51: RuntimeWarning:

invalid value encountered in divide

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:57: RuntimeWarning:

invalid value encountered in multiply

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:57: RuntimeWarning:

invalid value encountered in divide

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:61: RuntimeWarning:

invalid value encountered in multiply

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/4254463962.py:61: RuntimeWarning:

invalid value encountered in divide

/var/folders/cw/0t2jzw0d1dg1rj3qj57z9_qh0000gn/T/ipykernel_98750/3043756984.py:60: RuntimeWarning:

invalid value encountered in divide



> *Simulate the Heston model under \(Q\) using log-Euler for \(S\) and full-truncation Euler for \(v\).*

#### Given $(S_n, v_n)$:


#### 1. Draw correlated Brownian increments

Draw  
$$
Z_1, Z_2 \sim \mathcal{N}(0,1),
$$
and set
$$
\Delta W_1 = \sqrt{\Delta t}\, Z_1,
$$
$$
\Delta W_2 = \sqrt{\Delta t}\left(\rho Z_1 + \sqrt{1-\rho^2}\, Z_2\right).
$$


#### 2. Truncate variance

$$
v_n^{+} = \max(v_n, 0).
$$



#### 3. Update variance (full-truncation Euler)

$$
v_{n+1}
=
v_n
+
\kappa(\theta - v_n^{+})\,\Delta t
+
\xi \sqrt{v_n^{+}}\, \Delta W_2.
$$



#### 4. Update stock price (log-Euler)

$$
S_{n+1}
=
S_n
\exp\!\left(
\left(r - \tfrac{1}{2} v_n^{+}\right)\Delta t
+
\sqrt{v_n^{+}}\, \Delta W_1
\right).
$$


In [53]:

def simulate_heston_paths(
    S0, v0, r, q, T, kappa, theta, sigma, rho,
    Z1, Z2
):
    """
    Z1, Z2 shape: (n_steps, n_paths)
    """
    n_steps, n_paths = Z1.shape
    dt = T / n_steps
    sqdt = np.sqrt(dt)

    x = np.full(n_paths, np.log(S0), dtype=np.float64)
    v = np.full(n_paths, v0, dtype=np.float64)

    sqrt_1mr2 = np.sqrt(max(1.0 - rho * rho, 0.0))

    for t in range(n_steps):
        z1 = Z1[t]
        z2 = Z2[t]

        dWv = sqdt * z1
        dWs = sqdt * (rho * z1 + sqrt_1mr2 * z2)

        v_pos = np.maximum(v, 0.0)

        v = v + kappa * (theta - v_pos) * dt + sigma * np.sqrt(v_pos) * dWv
        v = np.maximum(v, 0.0)

        x = x + (r - q - 0.5 * v_pos) * dt + np.sqrt(v_pos) * dWs

    return np.exp(x)


def heston_call_price_mc(
    S0, K, v0, r, q, T, kappa, theta, sigma, rho,
    Z1, Z2
):
    ST = simulate_heston_paths(
        S0, v0, r, q, T, kappa, theta, sigma, rho,
        Z1, Z2
    )
    payoff = np.maximum(ST - K, 0.0)
    return float(np.exp(-r * T) * payoff.mean())



In [ ]:
S0 = 100
K  = 100
r  = 0.05
q  = 0.0
T  = 1.0

v0    = 0.04
kappa = 2.0
theta = 0.04
sigma = 0.5
rho   = -0.7
n_steps = 252
n_paths = 50_000

rng = np.random.default_rng(123)
Z1 = rng.standard_normal((n_steps, n_paths))
Z2 = rng.standard_normal((n_steps, n_paths))



model = heston_call_price_mc(
    spot, K, v0, r, q, T,
    kappa, theta, sigma, rho,
    Z1, Z2
)
model



94.01074464258363

In [13]:
calls = calls.copy()

calls["mid"] = 0.5 * (calls["bid"] + calls["ask"])

#Posi tive bid & ask 
#Sufficient liquidity
#Remove deep ITM/OTM garbage
#Use mid price
calls = calls[
    (calls["bid"] > 0) & 
    (calls["ask"] > 0) &
    (calls["volume"] > 10) &
    (calls["openInterest"] > 100)
]

calls = calls[["strike", "mid"]]
calls.head()

,strike,mid
0,0.5,188.45
1,1.0,187.80
2,1.5,187.20
3,2.0,186.70
4,2.5,186.30


In [14]:
calls = calls[
    (calls["strike"] > 0.7 * spot) &
    (calls["strike"] < 1.3 * spot)
]
expiry_dt = datetime.strptime(expiry, "%Y-%m-%d").replace(tzinfo=timezone.utc)
today = datetime.now(timezone.utc)

T = (expiry_dt - today).total_seconds() / (365.0 * 24 * 3600)


In [15]:
expiry_dt = datetime.strptime(expiry, "%Y-%m-%d").replace(tzinfo=timezone.utc)
today = datetime.now(timezone.utc)

T = (expiry_dt - today).total_seconds() / (365.0 * 24 * 3600)


In [16]:
quotes = calls.copy()
quotes["T"] = T
quotes.rename(columns={"strike": "K"}, inplace=True)

quotes.head()


,K,mid,T
168,135.0,53.975,0.031293
169,136.0,52.800,0.031293
171,140.0,49.175,0.031293
173,144.0,45.050,0.031293
174,145.0,44.075,0.031293


In [17]:
# pricing for feww strikes 
for _, row in quotes.head(5).iterrows():
    K = float(row["K"])
    mkt = float(row["mid"])

    model = heston_call_price_mc_crn(
        spot, K, v0, r, q, T,
        kappa, theta, sigma, rho,
        Z1, Z2
    )

    print(f"K={K:.1f}  market={mkt:.3f}  model={model:.3f}")


K=135.0  market=53.975  model=54.085
K=136.0  market=52.800  model=53.087
K=140.0  market=49.175  model=49.093
K=144.0  market=45.050  model=45.099
K=145.0  market=44.075  model=44.101


#### &rarr; attempt to calibrate heston params to market price:

In [18]:
def objective(params, quotes, S0, r, q, Z1, Z2):
    v0, kappa, theta, sigma, rho = params

    # hard constraints
    if v0 <= 1e-6 or kappa <= 1e-6 or theta <= 1e-6 or sigma <= 1e-6:
        return 1e10
    if abs(rho) >= 0.999:
        return 1e10

    err = 0.0
    for _, row in quotes.iterrows():
        K = float(row["K"])
        T = float(row["T"])
        mkt = float(row["mid"])

        # IMPORTANT: Z1,Z2 were generated for a specific T via dt=T/n_steps
        # so for now: single expiry (fixed T) calibration
        model = heston_call_price_mc_crn(
            S0, K, v0, r, q, T, kappa, theta, sigma, rho,
            Z1, Z2
        )
        err += (model - mkt) ** 2

    return err


In [19]:
from scipy.optimize import minimize

x0 = np.array([0.04, 2.0, 0.04, 0.5, -0.7])

bounds = [
    (1e-4, 0.5),     # v0
    (0.1, 10.0),     # kappa
    (1e-4, 0.5),     # theta
    (0.05, 3.0),     # sigma
    (-0.99, 0.99),   # rho
]

res = minimize(
    objective,
    x0=x0,
    args=(quotes, S0, r, q, Z1, Z2),
    method="L-BFGS-B",
    bounds=bounds,
    options={"maxiter": 100}
)

print(res.success, res.message)
# print("params* =", res.x)
names = ["v0", "kappa", "theta", "sigma", "rho"]

print("Calibrated Heston parameters:")
for name, val in zip(names, res.x):
    print(f"  {name:>6s} = {val: .6f}")



True CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
Calibrated Heston parameters:
      v0 =  0.040000
   kappa =  2.000000
   theta =  0.040000
   sigma =  0.500000
     rho = -0.700000


In [ ]:

    

def build_market_iv_surface_moneyness( 
    ticker_symbol: str,
    m_grid = np.linspace(0.8, 1.2, 21),   # K/S grid
    min_oi: int = 50,
    max_expiries: int = 10,
    ):
    """for each expiry, we interpolate its IVs onto this same grid, instead of select comon strikes across the set of expiries"""

    ticker = yf.Ticker(ticker_symbol)
    spot = ticker.info["currentPrice"]
    expiries = list(ticker.options)[:max_expiries]

    val_date = pd.Timestamp.now(tz="UTC").normalize()

    maturities = []
    Z = []

    for expiry in expiries:
        exp_dt = pd.to_datetime(expiry).tz_localize("UTC")
        T = (exp_dt - val_date).total_seconds() / (365.0 * 24 * 3600)
        if T <= 1/365:
            continue

        chain = ticker.option_chain(expiry)
        calls = chain.calls.copy()

        calls = calls[
            calls["impliedVolatility"].notna()
            & (calls["openInterest"] >= min_oi)
        ]
        if calls.empty:
            continue

        strikes = calls["strike"].to_numpy()
        ivs = calls["impliedVolatility"].to_numpy()

        # map to moneyness grid K = m * S
        K_grid = m_grid * spot

     
        order = np.argsort(strikes)
        strikes_sorted = strikes[order]
        ivs_sorted = ivs[order]

        iv_interp = np.interp(K_grid, strikes_sorted, ivs_sorted, left=np.nan, right=np.nan)

        maturities.append(T)
        Z.append(iv_interp)

    maturities = np.array(maturities)
    Z = np.array(Z)

    # sort by maturity
    idx = np.argsort(maturities)
    maturities = maturities[idx]
    Z = Z[idx]

    strikes_grid = m_grid * spot
    return spot, strikes_grid, maturities, Z



In [49]:

spot, strikes_grid, maturities, Z= build_market_iv_surface_moneyness("NVDA")


In [ ]:

X, Y = np.meshgrid(strikes_grid, maturities) 

fig = go.Figure(
    data=[go.Surface(x=X, y=Y, z=Z, colorscale="Viridis")]
)

fig.update_layout(
    # title=f"{ticker_symbol} Implied Volatility Surface",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Time to Maturity (years)",
        zaxis_title="Implied Volatility",
    ),
    height=700,
)

fig.show()

In [51]:
def build_bs_surface_from_market_atm(strikes_grid, maturities, market_iv_surface, spot):
    # ATM defined as strike closest to spot
    j_atm = int(np.argmin(np.abs(strikes_grid - spot)))
    atm_by_T = market_iv_surface[:, j_atm]  # (nT,)

    bs_surface = np.repeat(atm_by_T[:, None], len(strikes_grid), axis=1)
    return bs_surface, atm_by_T


In [52]:
def build_heston_iv_surface_mc(
    spot,
    strikes_grid,
    maturities,
    r, q,
    v0, kappa, theta, sigma, rho,
    n_paths=50_000,
    n_steps=252,
    seed=123
):
    iv_model = np.zeros((len(maturities), len(strikes_grid)))

    for i, T in enumerate(maturities):
        # keep dt consistent vs T by scaling steps (optional)
        steps = max(10, int(n_steps * T))  # rough scaling; you can also fix steps
        for j, K in enumerate(strikes_grid):
            price = heston_call_price_mc(
                S0=spot, K=float(K), v0=v0, r=r, q=q, T=float(T),
                kappa=kappa, theta=theta, sigma=sigma, rho=rho,
                n_paths=n_paths, n_steps=steps, seed=seed
            )
            iv_model[i, j] = implied_vol_call(price, spot, float(K), r, q, float(T))

    return iv_model
